# Xavier Deep network version
- loss가 최저인 global minimize지접을 찾는 것이 network의 목표

* !BUT! 출발점에 따라 Local Minima 또는 Saddle Point에 빠져서 Global Minima를 찾지 못할 수 있음

=> 출발점에 영향을 많이 받으므로, 출발점 weight를 좋게 만들기 위함

    * Normalization -> 평균: 0, 분산: 1
    * Xavier -> 평균: 0, 분산: 2/(Channel_in + Channe_out)

###### 기존 코드와 동일하고 weight 부분만 변경하면 됨!

## import

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os
print(tf.__version__)

2.1.0


## Checkpoint function

In [2]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt :
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

## Data load & pre-processing function

In [3]:
def load_mnist() :
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]
    test_data = np.expand_dims(test_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]

    train_data, test_data = normalize(train_data, test_data)

    train_labels = to_categorical(train_labels, 10) # [N,] -> [N, 10]
    test_labels = to_categorical(test_labels, 10) # [N,] -> [N, 10]

    return train_data, train_labels, test_data, test_labels

def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0

    return train_data, test_data

## Performance function

In [4]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits, y_true=labels, 
                                                                   from_logits=True))
    return loss

def accuracy_fn(model, images, labels):
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

## Model function

In [5]:
def flatten() :
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init) :
    return tf.keras.layers.Dense(units=label_dim, use_bias=True, kernel_initializer=weight_init)

def relu() :
    return tf.keras.layers.Activation(tf.keras.activations.relu)

## Create model (class version)

In [6]:
class create_model_class(tf.keras.Model):
    def __init__(self, label_dim):
        super(create_model_class, self).__init__()
        weight_init = tf.keras.initializers.glorot_uniform()

        self.model = tf.keras.Sequential()
        self.model.add(flatten())

        for i in range(4):
            self.model.add(dense(512, weight_init))
            self.model.add(relu())

        self.model.add(dense(label_dim, weight_init))

    def call(self, x, training=None, mask=None):

        x = self.model(x)

        return x

## Create model (function version)

In [7]:
def create_model_function(label_dim) :
    weight_init = tf.keras.initializers.glorot_uniform()

    model = tf.keras.Sequential()
    model.add(flatten())

    for i in range(4) :
        model.add(dense(512, weight_init))
        model.add(relu())

    model.add(dense(label_dim, weight_init))

    return model

## Define data & hyper-parameter

In [8]:
""" dataset """
train_x, train_y, test_x, test_y = load_mnist()

""" parameters """
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

""" Graph Input using Dataset API """
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=batch_size).\
    batch(batch_size, drop_remainder=True)

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=len(test_x)).\
    batch(len(test_x))

## Define model & optimizer & writer

In [10]:
""" Model """
network = create_model_function(label_dim)

""" Training """
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

""" Writer """
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_deep'

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

## Restore checkpoint & start train or test phase

In [11]:
if train_flag :

    checkpoint = tf.train.Checkpoint(dnn=network)

    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer(logdir=logs_dir)
    start_time = time()

    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)    

    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)        
        counter = checkpoint_counter        
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
    
    # train phase
    with summary_writer.as_default():  # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx, (train_input, train_label) in enumerate(train_dataset):            
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables))

                train_loss = loss_fn(network, train_input, train_label)
                train_accuracy = accuracy_fn(network, train_input, train_label)
                
                for test_input, test_label in test_dataset:                
                    test_accuracy = accuracy_fn(network, test_input, test_label)

                tf.summary.scalar(name='train_loss', data=train_loss, step=counter)
                tf.summary.scalar(name='train_accuracy', data=train_accuracy, step=counter)
                tf.summary.scalar(name='test_accuracy', data=test_accuracy, step=counter)

                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy))
                counter += 1                
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))
        
# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:    
        test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
 [!] Load failed...
Epoch: [ 0] [    0/  468] time: 1.6222, train_loss: 2.00058079, train_accuracy: 0.6250, test_Accuracy: 0.3728
Epoch: [ 0] [    1/  468] time: 2.3094, train_loss: 1.99337125, train_accuracy: 0.4844, test_Accuracy: 0.4304
Epoch: [ 0] [    2/  468] time: 3.1083, train_loss: 1.80028272, train_accuracy: 0.5781, test_Accuracy: 0.5229
Epoch: [ 0] [    3/  468] time: 4.0158, train_loss: 1.56165195, train_accuracy: 0.7422, test_Accuracy: 0.6615
Epoch: [ 0] [    4/  468] time: 4.8386, train_loss: 1.33682263, train_accuracy: 0.7500, test_Accuracy: 0.6926
Epoch: [ 0] [    5/  468] time: 5.5777, train_loss: 1.08556604, train_accuracy: 0.6953, test_Accuracy: 0.7084
Epoch: [ 0] [    6/  468] time: 6.2519, train_loss: 0.80517292, train_accuracy: 0.7500, test_Accuracy: 0.7126
Epoch: [ 0] [    7/  468] time: 6.8971, train_loss: 0.76341200, train_accuracy: 0.7344, test_Accuracy: 0.7321
Epoch: [ 0] [    8/  468] time: 7.5993,

Epoch: [ 0] [   74/  468] time: 54.1389, train_loss: 0.15109377, train_accuracy: 0.9844, test_Accuracy: 0.9342
Epoch: [ 0] [   75/  468] time: 54.8582, train_loss: 0.16386697, train_accuracy: 0.9453, test_Accuracy: 0.9327
Epoch: [ 0] [   76/  468] time: 55.8933, train_loss: 0.18131360, train_accuracy: 0.9375, test_Accuracy: 0.9294
Epoch: [ 0] [   77/  468] time: 57.2256, train_loss: 0.18036866, train_accuracy: 0.9297, test_Accuracy: 0.9275
Epoch: [ 0] [   78/  468] time: 58.6669, train_loss: 0.23372306, train_accuracy: 0.9297, test_Accuracy: 0.9280
Epoch: [ 0] [   79/  468] time: 59.7600, train_loss: 0.22739190, train_accuracy: 0.9141, test_Accuracy: 0.9298
Epoch: [ 0] [   80/  468] time: 61.4593, train_loss: 0.17368805, train_accuracy: 0.9531, test_Accuracy: 0.9324
Epoch: [ 0] [   81/  468] time: 62.9264, train_loss: 0.24795260, train_accuracy: 0.9375, test_Accuracy: 0.9311
Epoch: [ 0] [   82/  468] time: 64.0683, train_loss: 0.16224602, train_accuracy: 0.9531, test_Accuracy: 0.9303
E

Epoch: [ 0] [  148/  468] time: 124.0478, train_loss: 0.24930125, train_accuracy: 0.9219, test_Accuracy: 0.9467
Epoch: [ 0] [  149/  468] time: 124.9155, train_loss: 0.09076965, train_accuracy: 0.9844, test_Accuracy: 0.9484
Epoch: [ 0] [  150/  468] time: 125.9468, train_loss: 0.24398835, train_accuracy: 0.9219, test_Accuracy: 0.9476
Epoch: [ 0] [  151/  468] time: 126.8802, train_loss: 0.09797139, train_accuracy: 0.9766, test_Accuracy: 0.9429
Epoch: [ 0] [  152/  468] time: 127.7122, train_loss: 0.15856653, train_accuracy: 0.9375, test_Accuracy: 0.9399
Epoch: [ 0] [  153/  468] time: 128.5524, train_loss: 0.19309285, train_accuracy: 0.9453, test_Accuracy: 0.9393
Epoch: [ 0] [  154/  468] time: 129.5952, train_loss: 0.14244214, train_accuracy: 0.9609, test_Accuracy: 0.9381
Epoch: [ 0] [  155/  468] time: 130.4565, train_loss: 0.17352569, train_accuracy: 0.9609, test_Accuracy: 0.9383
Epoch: [ 0] [  156/  468] time: 131.3293, train_loss: 0.13582310, train_accuracy: 0.9453, test_Accuracy:

Epoch: [ 0] [  222/  468] time: 197.4153, train_loss: 0.10699195, train_accuracy: 0.9688, test_Accuracy: 0.9581
Epoch: [ 0] [  223/  468] time: 198.4866, train_loss: 0.12487249, train_accuracy: 0.9531, test_Accuracy: 0.9579
Epoch: [ 0] [  224/  468] time: 199.3607, train_loss: 0.13105088, train_accuracy: 0.9609, test_Accuracy: 0.9593
Epoch: [ 0] [  225/  468] time: 200.2596, train_loss: 0.27898407, train_accuracy: 0.9453, test_Accuracy: 0.9581
Epoch: [ 0] [  226/  468] time: 201.2911, train_loss: 0.12726326, train_accuracy: 0.9688, test_Accuracy: 0.9563
Epoch: [ 0] [  227/  468] time: 202.2666, train_loss: 0.11715915, train_accuracy: 0.9453, test_Accuracy: 0.9554
Epoch: [ 0] [  228/  468] time: 203.1730, train_loss: 0.22692385, train_accuracy: 0.9453, test_Accuracy: 0.9560
Epoch: [ 0] [  229/  468] time: 204.1440, train_loss: 0.20538113, train_accuracy: 0.9453, test_Accuracy: 0.9578
Epoch: [ 0] [  230/  468] time: 205.3040, train_loss: 0.13912804, train_accuracy: 0.9609, test_Accuracy:

Epoch: [ 0] [  296/  468] time: 269.6944, train_loss: 0.11055239, train_accuracy: 0.9766, test_Accuracy: 0.9635
Epoch: [ 0] [  297/  468] time: 270.8263, train_loss: 0.17394048, train_accuracy: 0.9531, test_Accuracy: 0.9615
Epoch: [ 0] [  298/  468] time: 271.7786, train_loss: 0.09550404, train_accuracy: 0.9688, test_Accuracy: 0.9566
Epoch: [ 0] [  299/  468] time: 272.7216, train_loss: 0.17947204, train_accuracy: 0.9453, test_Accuracy: 0.9571
Epoch: [ 0] [  300/  468] time: 273.6434, train_loss: 0.08620228, train_accuracy: 0.9844, test_Accuracy: 0.9575
Epoch: [ 0] [  301/  468] time: 274.6743, train_loss: 0.16881199, train_accuracy: 0.9453, test_Accuracy: 0.9594
Epoch: [ 0] [  302/  468] time: 275.5757, train_loss: 0.16182294, train_accuracy: 0.9688, test_Accuracy: 0.9597
Epoch: [ 0] [  303/  468] time: 276.4678, train_loss: 0.05991532, train_accuracy: 0.9766, test_Accuracy: 0.9619
Epoch: [ 0] [  304/  468] time: 277.6772, train_loss: 0.06981210, train_accuracy: 0.9688, test_Accuracy:

Epoch: [ 0] [  370/  468] time: 344.0159, train_loss: 0.14529778, train_accuracy: 0.9609, test_Accuracy: 0.9615
Epoch: [ 0] [  371/  468] time: 344.9786, train_loss: 0.14118911, train_accuracy: 0.9609, test_Accuracy: 0.9631
Epoch: [ 0] [  372/  468] time: 345.9458, train_loss: 0.06095782, train_accuracy: 0.9922, test_Accuracy: 0.9607
Epoch: [ 0] [  373/  468] time: 346.9602, train_loss: 0.09958024, train_accuracy: 0.9609, test_Accuracy: 0.9542
Epoch: [ 0] [  374/  468] time: 347.8748, train_loss: 0.17222315, train_accuracy: 0.9453, test_Accuracy: 0.9547
Epoch: [ 0] [  375/  468] time: 348.8755, train_loss: 0.12807232, train_accuracy: 0.9922, test_Accuracy: 0.9568
Epoch: [ 0] [  376/  468] time: 349.9658, train_loss: 0.08656075, train_accuracy: 0.9844, test_Accuracy: 0.9600
Epoch: [ 0] [  377/  468] time: 350.9493, train_loss: 0.08477349, train_accuracy: 0.9766, test_Accuracy: 0.9637
Epoch: [ 0] [  378/  468] time: 351.9316, train_loss: 0.08832372, train_accuracy: 0.9766, test_Accuracy:

Epoch: [ 0] [  444/  468] time: 416.7594, train_loss: 0.05743033, train_accuracy: 0.9922, test_Accuracy: 0.9651
Epoch: [ 0] [  445/  468] time: 417.7069, train_loss: 0.08359405, train_accuracy: 0.9688, test_Accuracy: 0.9641
Epoch: [ 0] [  446/  468] time: 418.7593, train_loss: 0.08106790, train_accuracy: 0.9922, test_Accuracy: 0.9639
Epoch: [ 0] [  447/  468] time: 419.8645, train_loss: 0.15403111, train_accuracy: 0.9453, test_Accuracy: 0.9635
Epoch: [ 0] [  448/  468] time: 420.8035, train_loss: 0.09170921, train_accuracy: 0.9766, test_Accuracy: 0.9628
Epoch: [ 0] [  449/  468] time: 421.7493, train_loss: 0.06199056, train_accuracy: 0.9844, test_Accuracy: 0.9623
Epoch: [ 0] [  450/  468] time: 422.8224, train_loss: 0.05475232, train_accuracy: 0.9844, test_Accuracy: 0.9604
Epoch: [ 0] [  451/  468] time: 423.6855, train_loss: 0.16664217, train_accuracy: 0.9609, test_Accuracy: 0.9592
Epoch: [ 0] [  452/  468] time: 424.6585, train_loss: 0.09588683, train_accuracy: 0.9688, test_Accuracy: